In [ ]:
import time
start = time.time()

In [ ]:
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import ast
import sys
from tqdm import tqdm


In [ ]:
sys.path.append('../input/yolov5reef/yolov5')
from custom_utils import plot_bboxes, get_ensemble, ensemble_predict
!mkdir -p /root/.config/Ultralytics
!cp -r /kaggle/input/yolov5reef/Arial.ttf /root/.config/Ultralytics/Arial.ttf

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env() # initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

# ensemble & tracking

In [ ]:
from IPython.display import clear_output
!pip install ../input/tracking/filterpy-1.4.5-py3-none-any.whl
!pip install ../input/tracking/commonmark-0.9.1-py2.py3-none-any.whl
!pip install ../input/tracking/rich-9.13.0-py3-none-any.whl
!pip install ../input/tracking/norfair-0.3.1-py3-none-any.whl
clear_output()

from norfair import Detection, Tracker


In [ ]:
def ensemble_to_norfair(prediction, frame_id):
    result = []
    for xc, yc, w, h, score in prediction:
        result.append(Detection(points=np.array([int(xc), int(yc)]), scores=np.array([score]), data=np.array([int(w), int(h), frame_id])))
    return result

def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

test = pd.read_csv('../input/tensorflow-great-barrier-reef/test.csv')

In [ ]:
MODELS =[
    ['../input/yolov5weights3/yolov5l6_val2_best.pt', 2880*2, 0.1, False],
    ['../input/reefyolov5fulltrainweights/yolov5x6_2400_seq_id_rand_0.pt',2400*1.5, 0.1, False],
    [ '../input/yolov5seqweights0/yolov5l6_seq_3008_22_f2_best.pt', 3008*1.5, 0.1, False],
    ['../input/reefyolov5fulltrainweights/epoch6.pt', 3008*1.5, 0.1, False],
    ['../input/reefyolov5fulltrainweights/yolov5l6_2880_seq_id_6_best.pt', 2880,0.1, False]
]

models, MODEL_PATHS, IMG_SIZES, CONF_THRESHS = get_ensemble(MODELS, 0.4)

# test

In [ ]:
start_pred = time.time()
final_thres = 0.37
wbf_skip_box_thr = 0.1
wbf_iou_thres = 0.35
conf_type = "max"
AUGMENT = [x[3] for x in MODELS]
TRACK_THRES = 0.35
hit_inertia_min = 3
hit_inertia_max = 10
initialization_delay = 3
distance_threshold = 35

tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=distance_threshold,
    hit_inertia_min=hit_inertia_min,
    hit_inertia_max=hit_inertia_max,
    initialization_delay=initialization_delay
)
seq_init = test.loc[0, 'sequence']
for n, (pixel_array, sample_prediction_df) in enumerate(tqdm(iter_test)):
    one_start = time.time()
    seq = test.loc[n, 'sequence']
    if seq!=seq_init:
        tracker = Tracker(
            distance_function=euclidean_distance, 
            distance_threshold=distance_threshold,
            hit_inertia_min=hit_inertia_min,
            hit_inertia_max=hit_inertia_max,
            initialization_delay=initialization_delay
            )
        seq_init = seq
    final_bboxes, tracks = ensemble_predict(
        models = models,
        img_sizes = IMG_SIZES,
        img = pixel_array,
        augment = AUGMENT,
        skip_box_thr = wbf_skip_box_thr,
        iou_thres = wbf_iou_thres,
        final_thres = final_thres,
        conf_type = conf_type,
        mod_weights=None,
        WBF = True
    )
    tracked_objects = tracker.update(detections=ensemble_to_norfair(tracks, n))
    for obj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = obj.last_detection.data
        if last_detected_frame_id == n or (n - last_detected_frame_id)==5:
            continue
        else:
            score = obj.last_detection.scores[0]
            if score>=TRACK_THRES:
                xc, yc = obj.estimate[0]
                x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
                score *= 0.5
                new_bbox_width = bbox_width if (x_min + bbox_width)<=1280 else (1280-x_min)
                new_bbox_height = bbox_height if (y_min + bbox_height)<=720 else (720-y_min)
                if x_min < 0:
                    new_bbox_width = new_bbox_width + x_min
                    x_min = 0
                if y_min < 0:
                    new_bbox_height = new_bbox_height + y_min
                    y_min = 0
                if bbox_width*bbox_height >= 280 and new_bbox_height >=np.max([bbox_height*0.3, 18]) and new_bbox_width>=np.max([bbox_width*0.3,18]):
                    final_bboxes.append('{:.3f} {} {} {} {}'.format(score, x_min, y_min, new_bbox_width, new_bbox_height))
    sample_prediction_df["annotations"] = ' '.join(final_bboxes)
    env.predict(sample_prediction_df)
    time.time() - one_start
submission = pd.read_csv('/kaggle/working/submission.csv')
submission.head()